# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 4 2023 8:30AM,254196,10,0086280186,ISDIN Corporation,Released
1,Jan 4 2023 8:30AM,254196,10,0086280187,ISDIN Corporation,Released
2,Jan 4 2023 8:30AM,254196,10,0086280188,ISDIN Corporation,Released
3,Jan 4 2023 8:30AM,254196,10,0086280409,ISDIN Corporation,Released
4,Jan 4 2023 8:30AM,254196,10,0086280521,ISDIN Corporation,Released
5,Jan 4 2023 8:30AM,254196,10,0086280520,ISDIN Corporation,Released
6,Jan 4 2023 8:30AM,254196,10,0086280522,ISDIN Corporation,Released
7,Jan 4 2023 8:30AM,254196,10,0086280524,ISDIN Corporation,Released
8,Jan 4 2023 8:30AM,254196,10,0086280523,ISDIN Corporation,Released
9,Jan 4 2023 8:30AM,254196,10,0086280177,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
48,254209,Released,1
49,254210,Released,1
50,254211,Released,3
51,254212,Released,1
52,254214,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
254209,NaN,NaN,1.0
254210,NaN,NaN,1.0
254211,NaN,NaN,3.0
254212,NaN,NaN,1.0
254214,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254104,0.0,0.0,1.0
254108,0.0,1.0,23.0
254121,0.0,41.0,9.0
254123,0.0,38.0,21.0
254124,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254104,0,0,1
254108,0,1,23
254121,0,41,9
254123,0,38,21
254124,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254104,0,0,1
1,254108,0,1,23
2,254121,0,41,9
3,254123,0,38,21
4,254124,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254104,,,1
1,254108,,1,23
2,254121,,41,9
3,254123,,38,21
4,254124,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 4 2023 8:30AM,254196,10,ISDIN Corporation
17,Jan 4 2023 8:30AM,254211,10,"Citieffe, Inc."
20,Jan 4 2023 8:30AM,254207,10,Livs Products
21,Jan 3 2023 3:25PM,254214,10,"Nextsource Biotechnology, LLC"
22,Jan 3 2023 3:18PM,254212,19,"Innogenix, LLC"
23,Jan 3 2023 3:11PM,254210,21,"NBTY Global, Inc."
24,Jan 3 2023 3:11PM,254209,21,"NBTY Global, Inc."
25,Jan 3 2023 3:09PM,254208,19,"AdvaGen Pharma, Ltd"
26,Jan 3 2023 2:57PM,254206,19,"AdvaGen Pharma, Ltd"
43,Jan 3 2023 2:56PM,254205,10,MedStone Pharma LLC


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 4 2023 8:30AM,254196,10,ISDIN Corporation,,,17
1,Jan 4 2023 8:30AM,254211,10,"Citieffe, Inc.",,,3
2,Jan 4 2023 8:30AM,254207,10,Livs Products,,,1
3,Jan 3 2023 3:25PM,254214,10,"Nextsource Biotechnology, LLC",,,1
4,Jan 3 2023 3:18PM,254212,19,"Innogenix, LLC",,,1
5,Jan 3 2023 3:11PM,254210,21,"NBTY Global, Inc.",,,1
6,Jan 3 2023 3:11PM,254209,21,"NBTY Global, Inc.",,,1
7,Jan 3 2023 3:09PM,254208,19,"AdvaGen Pharma, Ltd",,,1
8,Jan 3 2023 2:57PM,254206,19,"AdvaGen Pharma, Ltd",,,17
9,Jan 3 2023 2:56PM,254205,10,MedStone Pharma LLC,,,8


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 4 2023 8:30AM,254196,10,ISDIN Corporation,17,,
1,Jan 4 2023 8:30AM,254211,10,"Citieffe, Inc.",3,,
2,Jan 4 2023 8:30AM,254207,10,Livs Products,1,,
3,Jan 3 2023 3:25PM,254214,10,"Nextsource Biotechnology, LLC",1,,
4,Jan 3 2023 3:18PM,254212,19,"Innogenix, LLC",1,,
5,Jan 3 2023 3:11PM,254210,21,"NBTY Global, Inc.",1,,
6,Jan 3 2023 3:11PM,254209,21,"NBTY Global, Inc.",1,,
7,Jan 3 2023 3:09PM,254208,19,"AdvaGen Pharma, Ltd",1,,
8,Jan 3 2023 2:57PM,254206,19,"AdvaGen Pharma, Ltd",17,,
9,Jan 3 2023 2:56PM,254205,10,MedStone Pharma LLC,8,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 4 2023 8:30AM,254196,10,ISDIN Corporation,17,,
1,Jan 4 2023 8:30AM,254211,10,"Citieffe, Inc.",3,,
2,Jan 4 2023 8:30AM,254207,10,Livs Products,1,,
3,Jan 3 2023 3:25PM,254214,10,"Nextsource Biotechnology, LLC",1,,
4,Jan 3 2023 3:18PM,254212,19,"Innogenix, LLC",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 4 2023 8:30AM,254196,10,ISDIN Corporation,17.0,NaN,NaN
1,Jan 4 2023 8:30AM,254211,10,"Citieffe, Inc.",3.0,NaN,NaN
2,Jan 4 2023 8:30AM,254207,10,Livs Products,1.0,NaN,NaN
3,Jan 3 2023 3:25PM,254214,10,"Nextsource Biotechnology, LLC",1.0,NaN,NaN
4,Jan 3 2023 3:18PM,254212,19,"Innogenix, LLC",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 4 2023 8:30AM,254196,10,ISDIN Corporation,17.0,0.0,0.0
1,Jan 4 2023 8:30AM,254211,10,"Citieffe, Inc.",3.0,0.0,0.0
2,Jan 4 2023 8:30AM,254207,10,Livs Products,1.0,0.0,0.0
3,Jan 3 2023 3:25PM,254214,10,"Nextsource Biotechnology, LLC",1.0,0.0,0.0
4,Jan 3 2023 3:18PM,254212,19,"Innogenix, LLC",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,6862512,431.0,100.0,2.0
12,508302,0.0,21.0,0.0
15,1016697,95.0,31.0,0.0
19,2287771,77.0,3.0,0.0
20,508253,2.0,0.0,0.0
21,762563,2.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,6862512,431.0,100.0,2.0
1,12,508302,0.0,21.0,0.0
2,15,1016697,95.0,31.0,0.0
3,19,2287771,77.0,3.0,0.0
4,20,508253,2.0,0.0,0.0
5,21,762563,2.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,431.0,100.0,2.0
1,12,0.0,21.0,0.0
2,15,95.0,31.0,0.0
3,19,77.0,3.0,0.0
4,20,2.0,0.0,0.0
5,21,2.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,431.0
1,12,Released,0.0
2,15,Released,95.0
3,19,Released,77.0
4,20,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,20,21
Status,,,,,,
Completed,2.0,0.0,0.0,0.0,0.0,0.0
Executing,100.0,21.0,31.0,3.0,0.0,1.0
Released,431.0,0.0,95.0,77.0,2.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,20,21
0,Completed,2.0,0.0,0.0,0.0,0.0,0.0
1,Executing,100.0,21.0,31.0,3.0,0.0,1.0
2,Released,431.0,0.0,95.0,77.0,2.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,20,21
0,Completed,2.0,0.0,0.0,0.0,0.0,0.0
1,Executing,100.0,21.0,31.0,3.0,0.0,1.0
2,Released,431.0,0.0,95.0,77.0,2.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()